In [1]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

READER_MODEL_NAME = "NousResearch/Hermes-2-Pro-Llama-3-8B"

bnb_config = BitsAndBytesConfig(
     load_in_4bit=True,
     bnb_4bit_use_double_quant=True,
     bnb_4bit_quant_type="nf4",
     bnb_4bit_compute_dtype=torch.bfloat16,
 )
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME ,quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500
  #  device="cuda:0",
)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f6bbcf2eb50>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
2024-10-07 09:48:23.352002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-07 09:48:23.370720: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-07 09:48:23.376523: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-07 09:48:

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [18]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """- Read the given JSON structure and provide only the answer to the following questions without explaining how it was calculated .
        - Provide concise, direct answers without additional explanations.
        - Not imagining any additional information.
        - if unclear question. Ask for clarification.
        - Tell user if can't answer precisely or missing information.
        
        """,
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

context = '''\nExtracted documents:
{
  "customer_name": "สมชาย ใจดี",
  "customer_id": "CUST12345",
  "expenses": [
    {
      "year": 2024,
      "cloud_provider": "AWS",
      "months": {
        "6": {
          "total": 12000,
          "products": [
            {
              "name": "EC2",
              "amount": 6000
            },
            {
              "name": "S3",
              "amount": 3000
            },
            {
              "name": "Lambda",
              "amount": 3000
            }
          ]
        },
        "7": {
          "total": 10000,
          "products": [
            {
              "name": "EC2",
              "amount": 5000
            },
            {
              "name": "RDS",
              "amount": 2500
            },
            {
              "name": "CloudFront",
              "amount": 2500
            }
          ]
        },
        "8": {
          "total": 13000,
          "products": [
            {
              "name": "EC2",
              "amount": 7000
            },
            {
              "name": "S3",
              "amount": 4000
            },
            {
              "name": "Lambda",
              "amount": 2000
            }
          ]
        },
        "9": {
          "total": 14000,
          "products": [
            {
              "name": "EC2",
              "amount": 8000
            },
            {
              "name": "S3",
              "amount": 4000
            },
            {
              "name": "RDS",
              "amount": 2000
            }
          ]
        }
      }
    },
    {
      "year": 2024,
      "cloud_provider": "Google Cloud",
      "months": {
        "6": {
          "total": 11000,
          "products": [
            {
              "name": "Compute Engine",
              "amount": 6000
            },
            {
              "name": "BigQuery",
              "amount": 3000
            },
            {
              "name": "Cloud Storage",
              "amount": 2000
            }
          ]
        },
        "7": {
          "total": 9000,
          "products": [
            {
              "name": "Compute Engine",
              "amount": 4000
            },
            {
              "name": "Cloud Functions",
              "amount": 3000
            },
            {
              "name": "Cloud Storage",
              "amount": 2000
            }
          ]
        },
        "8": {
          "total": 12000,
          "products": [
            {
              "name": "Compute Engine",
              "amount": 6000
            },
            {
              "name": "BigQuery",
              "amount": 4000
            },
            {
              "name": "Cloud Storage",
              "amount": 2000
            }
          ]
        },
        "9": {
          "total": 13000,
          "products": [
            {
              "name": "Compute Engine",
              "amount": 7000
            },
            {
              "name": "BigQuery",
              "amount": 4000
            },
            {
              "name": "Cloud Functions",
              "amount": 2000
            }
          ]
        }
      }
    }
  ],
  "address": "123/45 ถ. สุขุมวิท กรุงเทพฯ 10110",
  "cloud_providers": [
    "AWS",
    "Google Cloud"
  ]
}
\n'''
# final_prompt = RAG_PROMPT_TEMPLATE.format(question="Cost of AWS on january", context=context)
# final_prompt = RAG_PROMPT_TEMPLATE.format(question="ค่าใช้จ่ายเดือนสิงหาคม เท่าไร", context=context)
# final_prompt = RAG_PROMPT_TEMPLATE.format(question="ค่าใช้จ่ายเดือนสิงหาคม product ไหนสูงสุด เท่าไร", context=context)
final_prompt = RAG_PROMPT_TEMPLATE.format(question="total cost of 2024", context=context)
# Redact an answer
answer = READER_LLM(final_prompt)[0]["generated_text"]
print(answer)

<|begin_of_text|><|im_start|>system
- Read the given JSON structure and provide only the answer to the following questions without explaining how it was calculated .
        - Provide concise, direct answers without additional explanations.
        - Not imagining any additional information.
        - if unclear question. Ask for clarification.
        - Tell user if can't answer precisely or missing information.
        
        <|im_end|>
<|im_start|>user
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}<|im_end|>
<|im_start|>assistant

The total cost of 2024 for AWS and Google Cloud is not directly provided in the given JSON structure. However, we can calculate the total cost by summing up the monthly costs for both cloud providers. 

For AWS:
Total cost = (June) 12000 + (July) 10000 + (August) 13000 + (September) 14000 = 49000

For Google Cloud:
Total cost = (June) 11000 + (July) 9000 + (August) 12000 + (September) 13000 = 36000

So, the comb